In [1]:
import pandas as pd
import glob
import requests
import json
import time

# Project notes

- Identify transactions that are missing from OC that are on any of the accounts (meaning banks, stripe, wise, paypal)
- Identify any transactions on the platform that are missing from the accounts
- Prep a balance/p&l with minimal wrangling

Project steps:

- Clarify project intention and goal (fill in project assessment in Make OC)
- Map out data 
- Write scripts 
- Analyze and prepare accounts

# Load config

In [2]:
# host variables
host = 'opensource'
dateFrom = '2022-12-31T22:59:59.999Z'
dateTo = '2023-12-31T22:59:59.999Z'
year = '2023'

In [3]:
# load API tokens from .env file
import os
from dotenv import load_dotenv
load_dotenv()
octoken = os.getenv('OCTOKEN')
wisetoken = os.getenv('WISETOKEN')

# set the profile id of the target Wise profile
wiseprofile = os.getenv('WISEPROFILE')

In [4]:
# Reload the data or not - set these to false after the first run to avoid reloading
reloadOC = False
reloadWise = False
reloadStripe = True

In [5]:
import locale
locale.getpreferredencoding()

'utf-8'

# Get data

In [6]:
# load transactions from open collective or from local file

def fetch_transactions():
    url = f"https://opencollective.com/api/graphql/v2?personalToken={octoken}"

    # Your GraphQL query
    query = """
    query TransactionsTable(
        $hostAccount: AccountReferenceInput,
        $limit: Int!,
        $offset: Int!,
        $type: TransactionType,
        $paymentMethodType: [PaymentMethodType],
        $dateFrom: DateTime,
        $dateTo: DateTime,
        $kind: [TransactionKind],
        $includeIncognitoTransactions: Boolean,
        $includeGiftCardTransactions: Boolean,
        $includeChildrenTransactions: Boolean,
        $virtualCard: [VirtualCardReferenceInput],
        $orderBy: ChronologicalOrderInput,
        $group: String,
        $includeHost: Boolean,
        $expense: ExpenseReferenceInput,
        $order: OrderReferenceInput
      ) {
        transactions(
          host: $hostAccount
          limit: $limit
          offset: $offset
          type: $type
          paymentMethodType: $paymentMethodType
          dateFrom: $dateFrom
          dateTo: $dateTo
          kind: $kind
          includeIncognitoTransactions: $includeIncognitoTransactions
          includeGiftCardTransactions: $includeGiftCardTransactions
          includeChildrenTransactions: $includeChildrenTransactions
          includeDebts: true
          virtualCard: $virtualCard
          orderBy: $orderBy
          group: $group
          includeHost: $includeHost
          expense: $expense
          order: $order
        ) {
          ...TransactionsTableQueryCollectionFragment
          __typename
        }
      }
      
      fragment TransactionsTableQueryCollectionFragment on TransactionCollection {
        totalCount
        offset
        limit
        nodes {
          id
          uuid
          kind
          amount {
            currency
            valueInCents
            __typename
          }
          amountInHostCurrency {
            currency
            valueInCents
            __typename
          }
          netAmount {
            currency
            valueInCents
            __typename
          }
          netAmountInHostCurrency {
            currency
            valueInCents
            __typename
          }
          paymentProcessorFee {
            currency
            valueInCents
            __typename
          }
          paymentMethod {
            name
            service
            sourcePaymentMethod {
              id
              name
              service
              __typename
            }
            type
          }
          payoutMethod {
            type
            name
            type
          }
          giftCardEmitterAccount {  
            id
            name
            slug
            __typename
          }
          group
          type
          description
          createdAt
          merchantId
          isRefunded
          isRefund
          refundTransaction {
            id
          }
          isOrderRejected
          account {
            ... on AccountWithParent {
              parent {
                id
                slug
                name
              }
            }
            id
            legacyId
            name
            slug
            isIncognito
            type
            __typename
          }
          oppositeAccount {
            id
            legacyId
            name
            slug
            isIncognito
            type
            __typename
          }
          expense {
            id
            type
            description
            invoiceInfo 
            tags
            payee {
              id
              name
              slug
              type
              __typename
            }
            __typename
          }
          permissions {
            id
            canRefund
            canDownloadInvoice
            canReject
            __typename
          }
          __typename
        }
        __typename
      }
    """

    headers = {
        "Authorization": f"Bearer {octoken}",
        "Content-Type": "application/json"
    }

    # Initial variables setup
    variables = {
        "hostAccount": {"slug": host},
        "includeIncognitoTransactions": True,
        "includeChildrenTransactions": True,
        "limit": 10000,  # Adjust if necessary but keep a sensible number to avoid server strain
        "offset": 0,  # Will be adjusted for each subsequent request
        "dateFrom": dateFrom,
        "dateTo": dateTo,
        "orderBy": {"field": "CREATED_AT", "direction": "DESC"},
        "includeHost": True
    }

    all_transactions = []  # To hold all transactions
    while True:
        
        # Try catch around json_data = response.json() to avoid errors

        max_retries = 10  # Setting the maximum number of retries
        retries = 0  # Initial retry count
        response = ''

        while retries < max_retries:
            try:
                # Make the HTTP request
                response = requests.post(
                    url, json={'query': query, 'variables': variables}, headers=headers)
                json_data = response.json()
                break 
            except Exception as e:
                print("Request failed. Attempt:", retries + 1)
                print(e)
                retries += 1 
                time.sleep(10)

        if retries == max_retries:
            print("Maximum retry attempts reached. Exiting.")
        # Extract data
        transactions = json_data['data']['transactions']['nodes']
        all_transactions.extend(transactions)

        # Pagination: Update offset
        variables['offset'] += variables['limit']

        # Check if all transactions are fetched
        if len(all_transactions) >= json_data['data']['transactions']['totalCount']:
            break
        
        # print progress
        print(f'Fetched {len(all_transactions)} transactions')
        # sleep for 10 seconds to avoid server strain
        time.sleep(5)

    return all_transactions

# if account has property parent, replace account with parent
def replace_account_with_parent(transaction):
  if 'parent' in transaction['account']:
    transaction['account'] = transaction['account']['parent']
  return transaction

# apply the function to all transactions
def post_process_transactions(all_transactions):
  return list(map(replace_account_with_parent, all_transactions))

if reloadOC:
  all_transactions = fetch_transactions()
  # dump all transactions to a json file
  post_process_transactions(all_transactions)
  with open(f'data/{host}/{year}/{host}_{year}_all_platform_transactions.json', 'w') as f:
      json.dump(all_transactions, f, indent=2)
  # convert the json file to a dataframe
  df_platform_transactions = pd.json_normalize(all_transactions)
  # save dataframe as pickle
  df_platform_transactions.to_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')
else:
  df_platform_transactions = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')


Fetched 10000 transactions
Fetched 20000 transactions
Fetched 30000 transactions
Fetched 40000 transactions
Fetched 50000 transactions
Fetched 60000 transactions
Fetched 70000 transactions
Fetched 80000 transactions
Fetched 90000 transactions
Fetched 100000 transactions
Fetched 110000 transactions
Fetched 120000 transactions
Fetched 130000 transactions
Fetched 140000 transactions
Fetched 150000 transactions
Fetched 160000 transactions
Fetched 170000 transactions
Fetched 180000 transactions
Fetched 190000 transactions
Fetched 200000 transactions
Fetched 210000 transactions
Fetched 220000 transactions
Fetched 230000 transactions
Fetched 240000 transactions
Fetched 250000 transactions
Fetched 260000 transactions
Fetched 270000 transactions
Fetched 280000 transactions
Fetched 290000 transactions
Fetched 300000 transactions
Fetched 310000 transactions
Fetched 320000 transactions
Fetched 330000 transactions
Request failed. Attempt: 1
Expecting value: line 1 column 1 (char 0)
Fetched 340000 t

In [8]:
# load transactions from wise

# Define the base URL for the Wise API
BASE_URL = 'https://api.transferwise.com/v1/transfers'

def fetch_all_transfers(profile_id=None, status=None, source_currency=None, 
                        target_currency=None, created_date_start=None, 
                        created_date_end=None, limit=100):
    headers = {
        'Authorization': f'Bearer {wisetoken}'
    }
    offset = 0
    all_transfers = []

    while True:
        # Prepare query parameters
        query_params = {'limit': limit, 'offset': offset}
        if profile_id:
            query_params['profile'] = profile_id
        if status:
            query_params['status'] = status
        if source_currency:
            query_params['sourceCurrency'] = source_currency
        if target_currency:
            query_params['targetCurrency'] = target_currency
        if created_date_start:
            query_params['createdDateStart'] = created_date_start
        if created_date_end:
            query_params['createdDateEnd'] = created_date_end

        # Make the GET request to the Wise API
        response = requests.get(BASE_URL, headers=headers, params=query_params)
        data = response.json()

        print(data)

        # Break if there are no more results
        if not data:
            break

        for transfer in data:
            all_transfers.append({
                'id': transfer['id'],
                'user': transfer['user'],
                'targetAccount': transfer['targetAccount'],
                'sourceAccount': transfer['sourceAccount'],
                'quote': transfer['quote'],
                'quoteUuid': transfer['quoteUuid'],
                'status': transfer['status'],
                'reference': transfer['reference'],
                'rate': transfer['rate'],
                'created': transfer['created'],
                'business': transfer['business'],
                'transferRequest': transfer['transferRequest'],
                'details.reference': transfer.get('details', {}).get('reference'),
                'hasActiveIssues': transfer['hasActiveIssues'],
                'sourceCurrency': transfer['sourceCurrency'],
                'sourceValue': transfer['sourceValue'],
                'targetCurrency': transfer['targetCurrency'],
                'targetValue': transfer['targetValue'],
                'customerTransactionId': transfer['customerTransactionId']
            })

        # Update the offset for the next iteration
        offset += limit

    return all_transfers

if reloadWise:
    transfers = fetch_all_transfers(profile_id=wiseprofile, created_date_start=dateFrom, created_date_end=dateTo)

    # create transfers dataframe
    df_wise_transactions = pd.DataFrame()

    for transfer in transfers:
        # add to dataframe with concat
        df_wise_transactions = pd.concat([df_wise_transactions, pd.DataFrame([transfer])])

    # save dataframe as pickle
    df_wise_transactions.to_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

In [9]:
# load transactions from stripe export
if reloadStripe:
    df_stripe_in = pd.read_csv(f'data/{host}/{year}/{host}_{year}_stripe.csv')
    # ensure only relevant columns
    df_stripe_in = df_stripe_in[[
    'id',
    'Created date (UTC)',
    'Description',
    'Seller Message',
    'Amount',
    'Amount Refunded',
    'Currency',
    'Converted Amount',
    'Converted Amount Refunded',
    'Fee',
    'Taxes On Fee',
    'Converted Currency',
    'Mode',
    'Status',
    'Statement Descriptor',
    'Customer ID',
    'Customer Description',
    'Captured',
    'Card ID',
    'PaymentIntent ID',
    'Application Fee',
    'Application ID',
    'from (metadata)',
    'to (metadata)'
    ]]
    df_stripe_in.to_pickle(f'data/{host}/{year}/df_{host}_{year}_stripe_transactions.pkl')
    


/tmp/ipykernel_2219676/1810662741.py:3: DtypeWarning: Columns (26,27,28,29,30,31,37,38,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stripe_in = pd.read_csv(f'data/{host}/{year}/{host}_{year}_stripe.csv')


In [10]:
# load open collective data
df_platform = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')

# load stripe data
df_stripe = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_stripe_transactions.pkl')

# load wise data
df_wise = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

# load bank data
dir = f'data/{host}/{year}/oscbank_1'
df_bank_1 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_1 = df_bank_1.dropna(axis=1, how='all')

dir = f'data/{host}/{year}/oscbank_2'
df_bank_2 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_2 = df_bank_2.dropna(axis=1, how='all')

# Post process data

In [11]:
# post process

# remove failed and pending transactions from Stripe
df_stripe = df_stripe[df_stripe['Status'] != 'Failed']
df_stripe = df_stripe[df_stripe['Status'] != 'Pending']

# remove cancelled from Wise
df_wise = df_wise[df_wise['status'] != 'cancelled']

# Inspect transactions dataframe

In [12]:
# get unique transaction kind
transaction_kinds = df_platform_transactions['kind'].unique()

# sample 2 transactions for each kind
df_sample = df_platform_transactions.groupby('kind').apply(lambda x: x.sample(2))

df_sample

id  \
kind                                                                         
ADDED_FUNDS                   596777  2dc4c418-f5c5-47a7-88f3-0f84f2a3b357   
                              667897  595e994a-de7e-4a86-89d8-09c148ea7456   
BALANCE_TRANSFER              655320  c82dfdbd-4939-4a9e-ad2b-d3ed42ca190e   
                              799996  3fa54306-3edd-4178-bfd9-8e8aba7de49b   
CONTRIBUTION                  618017  e8a571a5-187c-48be-aec0-6ccc9a6c9b51   
                              670346  965205dd-08d8-498c-a4ef-8d3466d894cd   
EXPENSE                       643441  76680a4d-e74c-47a5-b836-42283f914ff9   
                              510628  88bc4d90-f907-4f69-bdf4-ea2087b1fdb0   
HOST_FEE                      110435  5ef59acc-3fed-46b3-9f74-e0f80844f50e   
                              517140  bf902c22-7d87-42d6-aa8c-4138e06f95ac   
HOST_FEE_SHARE                785164  b0f820a4-7b53-4eb1-a3a8-dd9c98500b9d   
                              375924  01757300-7996-49da-a5b7-373499a89bda   
HOST_FEE_SHARE_DEBT           86844   0f0fe009-77fc-4c2e-8f3b-4d8965e5e0f3   
                              301320  b991dce8-de60-4b39-972b-b8f56214b7bc   
PAYMENT_PROCESSOR_COVER       81790   98d067e2-9e67-45d4-b660-db7e7d717c3a   
                              739039  5c3ab984-e558-45e2-a40f-4b8aac26b7e9   
PAYMENT_PROCESSOR_DISPUTE_FEE 296099  a037d537-ede6-43ea-81bc-704a2f01d520   
                              241325  eba512e4-8a80-4648-b804-1995ae393053   
PLATFORM_TIP_DEBT             665608  67c14665-63a5-4543-95df-738e595aec2f   
                              389150  10e6e73e-29e6-41db-a361-2921030d771b   
PREPAID_PAYMENT_METHOD        731485  a623102f-5327-4f5c-95f9-b15d282e9977   
                              159049  ed29b08c-2922-41ce-86fc-5f561fb0cbba   

                                                                      uuid  \
kind                                                                         
ADDED_FUNDS                   596777  2dc4c418-f5c5-47a7-88f3-0f84f2a3b357   
                              667897  595e994a-de7e-4a86-89d8-09c148ea7456   
BALANCE_TRANSFER              655320  c82dfdbd-4939-4a9e-ad2b-d3ed42ca190e   
                              799996  3fa54306-3edd-4178-bfd9-8e8aba7de49b   
CONTRIBUTION                  618017  e8a571a5-187c-48be-aec0-6ccc9a6c9b51   
                              670346  965205dd-08d8-498c-a4ef-8d3466d894cd   
EXPENSE                       643441  76680a4d-e74c-47a5-b836-42283f914ff9   
                              510628  88bc4d90-f907-4f69-bdf4-ea2087b1fdb0   
HOST_FEE                      110435  5ef59acc-3fed-46b3-9f74-e0f80844f50e   
                              517140  bf902c22-7d87-42d6-aa8c-4138e06f95ac   
HOST_FEE_SHARE                785164  b0f820a4-7b53-4eb1-a3a8-dd9c98500b9d   
                              375924  01757300-7996-49da-a5b7-373499a89bda   
HOST_FEE_SHARE_DEBT           86844   0f0fe009-77fc-4c2e-8f3b-4d8965e5e0f3   
                              301320  b991dce8-de60-4b39-972b-b8f56214b7bc   
PAYMENT_PROCESSOR_COVER       81790   98d067e2-9e67-45d4-b660-db7e7d717c3a   
                              739039  5c3ab984-e558-45e2-a40f-4b8aac26b7e9   
PAYMENT_PROCESSOR_DISPUTE_FEE 296099  a037d537-ede6-43ea-81bc-704a2f01d520   
                              241325  eba512e4-8a80-4648-b804-1995ae393053   
PLATFORM_TIP_DEBT             665608  67c14665-63a5-4543-95df-738e595aec2f   
                              389150  10e6e73e-29e6-41db-a361-2921030d771b   
PREPAID_PAYMENT_METHOD        731485  a623102f-5327-4f5c-95f9-b15d282e9977   
                              159049  ed29b08c-2922-41ce-86fc-5f561fb0cbba   

                                                               kind  \
kind                                                                  
ADDED_FUNDS                   596777                    ADDED_FUNDS   
                              667897                    ADDED_FUNDS   
BALANCE_TRANSFER              655320      

In [13]:
# show all columns
pd.set_option('display.max_columns', None)
df_platform.sample(10)

,id,uuid,kind,paymentMethod,payoutMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,payoutMethod.type,payoutMethod.name,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
657467,1d378403-fdc4-4440-ab97-1254d1dbca22,1d378403-fdc4-4440-ab97-1254d1dbca22,HOST_FEE_SHARE,NaN,NaN,NaN,826b4b89-b2fa-43e2-ac18-a872a131322f,DEBIT,Host Fee Share,2023-03-17T10:31:18.771Z,None,False,False,NaN,False,NaN,Debit,USD,-25,Amount,USD,-25,Amount,USD,-25,Amount,USD,-25,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3w-g860ej5l-avnodgx7,8686,Open Collective,opencollective,False,ORGANIZATION,Organization,gnxdwzj3-le5mpwmw-bdvqy8rv-bko04a97,False,True,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
837507,129f8cdb-a745-4fce-b508-2e41f143c025,129f8cdb-a745-4fce-b508-2e41f143c025,HOST_FEE,NaN,NaN,NaN,97449dad-76dc-48f0-b8c8-46f109132693,CREDIT,Host Fee,2023-01-01T05:04:28.841Z,None,False,False,NaN,False,NaN,Credit,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,4rxg0j35-lzkwm6v3-v5xpvoe9-8n47daby,139424,NativeScript,nativescript,False,COLLECTIVE,Collective,mywxoz34-09rl6k9m-mjbqvenb-dj7gk85a,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45022,ff31fed8-9cdd-476c-b65c-7959d02986be,ff31fed8-9cdd-476c-b65c-7959d02986be,HOST_FEE,NaN,NaN,NaN,0922f837-9878-4fee-952d-6e34c78f5eac,CREDIT,Host Fee,2023-12-02T07:03:18.632Z,None,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,500,Amount,USD,500,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,7mywxoz3-409rl6kg-wgnpvenb-dj7gk85a,13868,Framework7,framework7,False,COLLECTIVE,Collective,3kzxy4v0-7wlr6m0n-xympmj9n-o8agdbe5,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397500,7e7ab7b6-cb7a-47c9-bfc7-ab4275283650,7e7ab7b6-cb7a-47c9-bfc7-ab4275283650,CONTRIBUTION,NaN,NaN,NaN,b11d0710-4cc7-43fe-9d3b-32628f527170,CREDIT,Monthly financial contribution to Apple M1 bui...,2023-07-03T10:02:48.627Z,0H2728403H500332J,False,False,NaN,False,NaN,Credit,USD,1000,Amount,USD,1000,Amount,USD,901,Amount,USD,901,Amount,USD,-99,Amount,ggnxdwzj-3le5mpw7-4kjqy8rv-bko04a97,NaN,IfcOpenShell,opensourcebim,NaN,NaN,NaN,zaxon793-jy8gpl3b-3x9pbrkd-emwl5v04,539821,StefStap,stefstap,False,INDIVIDUAL,Individual,vedj9wro-z3a56dlj-5eap7blg-8x4m0ykn,True,True,True,Tran

In [14]:
# count transactions by kind
df_platform.groupby(['kind']).size()

kind
ADDED_FUNDS                        2616
BALANCE_TRANSFER                    116
CONTRIBUTION                     193827
EXPENSE                            7582
HOST_FEE                         385894
HOST_FEE_SHARE                   192946
HOST_FEE_SHARE_DEBT               61479
PAYMENT_PROCESSOR_COVER             728
PAYMENT_PROCESSOR_DISPUTE_FEE        18
PLATFORM_TIP_DEBT                    11
PREPAID_PAYMENT_METHOD               13
dtype: int64

# Reconciling Stripe

## Automatic reconciliation by merchantId

In [15]:
# copy df_strip to df_s
df_stripe_reconciliation = df_stripe.copy()

In [16]:
df_stripe_reconciliation.head()

,id,Created date (UTC),Description,Seller Message,Amount,Amount Refunded,Currency,Converted Amount,Converted Amount Refunded,Fee,Taxes On Fee,Converted Currency,Mode,Status,Statement Descriptor,Customer ID,Customer Description,Captured,Card ID,PaymentIntent ID,Application Fee,Application ID,from (metadata),to (metadata)
0,ch_3OTXgHBYycQg1OMf0IfKN8Sw,2023-12-31 22:46:55,Monthly financial contribution to Logseq (Back...,Payment complete.,5.00,0.0,usd,5.00,0.0,0.77,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,cus_PI7wZ2sLFYmMPi,https://opencollective.com/sanket-sharma,True,pm_1OTXmVBYycQg1OMfzXUwCvAm,pi_3OTXgHBYycQg1OMf0PkaNI3M,0.25,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,https://opencollective.com/sanket-sharma,https://opencollective.com/logseq
3,ch_3OTXVMBYycQg1OMf0qVZ5tjm,2023-12-31 22:29:41,Financial contribution to OpenSCAD,Payment complete.,50.00,0.0,usd,50.00,0.0,4.25,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTXVpBYycQg1OMfien2j8Cr,pi_3OTXVMBYycQg1OMf02f4Vhyl,2.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/openscad
4,ch_3OTULSBYycQg1OMf10wlDceK,2023-12-31 19:06:47,Monthly financial contribution to LibreELEC,Payment complete.,10.00,0.0,usd,10.00,0.0,1.24,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,cus_PI4UZkic5DVDO1,https://opencollective.com/daniel-dolejska,True,pm_1OTULSBYycQg1OMf7Lt0BwEi,pi_3OTULSBYycQg1OMf1saV7WWG,0.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,https://opencollective.com/daniel-dolejska,https://opencollective.com/libreelec
5,ch_3OTUG8BYycQg1OMf0FRuZULZ,2023-12-31 19:03:18,Financial contribution to Spotube,Payment complete.,5.00,0.0,usd,5.00,0.0,0.77,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTUI5BYycQg1OMfAbkDmXhS,pi_3OTUG8BYycQg1OMf0AcGFa2r,0.25,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/spotube
7,ch_3OTT8EBYycQg1OMf0gR3PzrY,2023-12-31 17:49:54,Financial contribution to .fmbot (.fmbot lifet...,Payment complete.,49.99,0.0,usd,49.99,0.0,4.25,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTT94BYycQg1OMf9AOHyAVx,pi_3OTT8EBYycQg1OMf0xdn31JV,2.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/fmbot


In [17]:
# df_platform where kind is CONTRIBUTION and paymentMethod.service is STRIPE
df_platform_stripe = df_platform[(df_platform['kind'] == 'CONTRIBUTION') & (df_platform['paymentMethod.service'] == 'STRIPE')]

In [18]:
df_platform_stripe.head()

,id,uuid,kind,paymentMethod,payoutMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,payoutMethod.type,payoutMethod.name,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
3,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,CONTRIBUTION,NaN,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,CREDIT,Monthly financial contribution to Logseq (Back...,2023-12-31T22:46:58.403Z,ch_3OTXgHBYycQg1OMf0IfKN8Sw,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,n4gx0bro-v5m96nkd-3ay6d8lk-3ey7jzwa,744511,Sanket Sharma,sanket-sharma,False,INDIVIDUAL,Individual,8k03reyd-5agmq5r7-rlyplbwo-z7j4nxv9,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,f3fb4940-1026-4440-bd85-bc3c0cb439b0,f3fb4940-1026-4440-bd85-bc3c0cb439b0,CONTRIBUTION,NaN,NaN,NaN,9931763a-79b1-4141-923f-216db174f56e,CREDIT,Financial contribution to OpenSCAD,2023-12-31T22:29:44.179Z,ch_3OTXVMBYycQg1OMf0qVZ5tjm,False,False,NaN,False,NaN,Credit,USD,5000,Amount,USD,5000,Amount,USD,4825,Amount,USD,4825,Amount,USD,-175,Amount,rmvrwng4-kj03dpbk-ljzpz57o-yl9e8xba,132610.0,OpenSCAD,openscad,False,COLLECTIVE,Collective,n4gx0bro-v5m96n0l-878qd8lk-3ey7jzwa,467683,Holland Hopson,guest-8761eb54,False,INDIVIDUAL,Individual,ywz9j4av-god8pg8g-8ly6mr35-nxklb0e7,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,e438b64b-473b-405b-be1c-67c7cfc58fba,e438b64b-473b-405b-be1c-67c7cfc58fba,CONTRIBUTION,NaN,NaN,NaN,91d79b1b-e38d-4559-be03-ad4a5f5fa301,CREDIT,Monthly financial contribution to LibreELEC,2023-12-31T19:06:49.150Z,ch_3OTULSBYycQg1OMf10wlDceK,False,False,NaN,False,NaN,Credit,USD,1000,Amount,USD,1000,Amount,USD,926,Amount,USD,926,Amount,USD,-74,Amount,88rzownx-l9e50pxv-3vb6ymvb-dgk7j43a,204839.0,LibreELEC,libreelec,False,COLLECTIVE,Collective,e0adkjrv-8xwm69oy-zwl67z5n-4l93bgoy,744470,Daniel Dolejska,daniel-dolejska,False,INDIVIDUAL,Individual,eng0kzdy-vor4pzez-e7bpbma8-37xlw95j,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,8ee3d5b0-b031-4b06-b7ab-69a8419900c2,8ee3d5b0-b031-4b06-b7ab-69a8419900c2,CONTRIBUTION,NaN,NaN,NaN,0870251c-c506-421d-b969-9b98f303bb90,CREDIT,Financial contribution to Spotube,2023-12-31T19:03:20.581Z,ch_3OTUG8BYycQg1OMf0FRuZULZ,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,3kzxy4v0-7wlr6mvy-5m76mj9n-o8agdbe5,475166.0,Spotube,spotube,False,COLLECTIV

In [19]:
# for each row in df_stripe, find the corresponding row in df_platform, match df_stripe['id'] with df_platform['merchantId']
# if the row exists, add the platform transaction id to df_stripe in a new column called platform_transaction_id
# if the row does not exist, set the platform_transaction_id to None

# Merge the two dataframes on the 'id' from df_stripe and 'merchantId' from df_platform
merged_df = pd.merge(df_stripe_reconciliation, df_platform[['merchantId', 'id']], left_on='id', right_on='merchantId', how='left', suffixes=('', '_platform_transaction'))

# Drop the extra 'merchantId' column from the merge
merged_df.drop(columns='merchantId', inplace=True)

# Assign the resulting dataframe back to df_stripe
df_stripe_reconciliation = merged_df

In [20]:
# count the number of Stripe transactions that have a id_platform_transaction
print(f"Stripe transactions that have a corresponding platform transaction: {len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].notnull()])}")

# count the number of Stripe transactions that do not have a id_platform_transaction
print(f"Stripe transactions without a corresponding platform transaction: {len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].isnull()])}")

# print the percentage of Stripe transactions that have a corresponding platform transaction
print(f"Percentage of Stripe transactions that have a corresponding platform transaction: {round(len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].notnull()])/len(df_stripe_reconciliation)*100, 2)}%")

Stripe transactions that have a corresponding platform transaction: 131588
Stripe transactions without a corresponding platform transaction: 432
Percentage of Stripe transactions that have a corresponding platform transaction: 99.67%


## Reconciling remaining Stripe transactions

In [21]:
df_stripe_remainder = df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].isnull()]

In [22]:
df_stripe_remainder.head()

,id,Created date (UTC),Description,Seller Message,Amount,Amount Refunded,Currency,Converted Amount,Converted Amount Refunded,Fee,Taxes On Fee,Converted Currency,Mode,Status,Statement Descriptor,Customer ID,Customer Description,Captured,Card ID,PaymentIntent ID,Application Fee,Application ID,from (metadata),to (metadata),id_platform_transaction
116,ch_3OSjH1BYycQg1OMf1SrSebC3,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMfAPu0eks5,pi_3OSjH1BYycQg1OMf1hLSOPbZ,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
117,ch_3OSjH1BYycQg1OMf03ruobeK,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMfOEXlem38,pi_3OSjH1BYycQg1OMf0l0hNQLu,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
118,ch_3OSjH1BYycQg1OMf0fkywX7k,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMftpAiBEfm,pi_3OSjH1BYycQg1OMf0QvSjaGu,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
119,ch_3OSjH0BYycQg1OMf04HatfLy,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMfqhqMfElP,pi_3OSjH0BYycQg1OMf0xkJN2s7,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
175,ch_3OSN05BYycQg1OMf1xfVvxkZ,2023-12-28 17:04:15,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSN0EBYycQg1OMf6jMM1Jzf,pi_3OSN05BYycQg1OMf13FUXY3w,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN


In [23]:
# count df_stripe_remainder by 'Status'
df_stripe_remainder.groupby(['Status']).size()

Status
Paid        143
Refunded    289
dtype: int64

In [24]:
# count df_stripe_remainder by 'Seller Message'
df_stripe_remainder.groupby(['Seller Message']).size()

Seller Message
One of your rules allowed this payment, overriding Stripe's risk evaluation and any address or CVC checks.     41
Payment complete.                                                                                             376
Stripe evaluated this payment as having elevated risk, and placed it in your manual review queue.              15
dtype: int64

In [25]:
# sum amount of df_stripe_remainder by status
df_stripe_remainder.groupby(['Status'])['Amount'].sum()

Status
Paid        3907.0
Refunded     293.0
Name: Amount, dtype: float64

In [26]:
# count df_stripe_remainder by 'Created date (UTC)' date only
# create a new column with the date only
df_stripe_remainder['Created date'] = pd.to_datetime(df_stripe_remainder['Created date (UTC)']).dt.date

df_stripe_remainder.groupby(['Created date']).size()

/tmp/ipykernel_2219676/2206905382.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stripe_remainder['Created date'] = pd.to_datetime(df_stripe_remainder['Created date (UTC)']).dt.date


Created date
2023-01-25     1
2023-03-13    10
2023-03-14    15
2023-03-15     2
2023-03-16     7
              ..
2023-06-29     8
2023-06-30     2
2023-12-27     2
2023-12-28     5
2023-12-29     4
Length: 73, dtype: int64

In [27]:
df_stripe_remainder.to_csv(f'data/missing_osc_stripe_transactions.csv')

In [28]:
# count df_stripe_remainder by 'to (metadata)'
df_stripe_remainder.groupby(['to (metadata)']).size()

to (metadata)
https://opencollective.com/advanced-bim-frontend-developer-course      1
https://opencollective.com/curl                                       37
https://opencollective.com/hledger                                     1
https://opencollective.com/manjaro-us                                 10
https://opencollective.com/mvvmcross                                   1
https://opencollective.com/openaddresses                             324
dtype: int64

In [29]:
df_platform.head()

,id,uuid,kind,paymentMethod,payoutMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,payoutMethod.type,payoutMethod.name,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
0,a985a0d8-c94e-4173-9918-3246e3231021,a985a0d8-c94e-4173-9918-3246e3231021,HOST_FEE_SHARE,NaN,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,DEBIT,Host Fee Share,2023-12-31T22:46:58.361Z,None,False,False,NaN,False,NaN,Debit,USD,-25,Amount,USD,-25,Amount,USD,-25,Amount,USD,-25,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3w-g860ej5l-avnodgx7,8686,Open Collective,opencollective,False,ORGANIZATION,Organization,vedj9wro-z3a56dzb-zllq7blg-8x4m0ykn,False,True,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,377540a1-80e4-45dd-8cf8-44e5a8d04f4d,377540a1-80e4-45dd-8cf8-44e5a8d04f4d,HOST_FEE,NaN,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,CREDIT,Host Fee,2023-12-31T22:46:58.361Z,None,False,False,NaN,False,NaN,Credit,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914,Logseq,logseq,False,COLLECTIVE,Collective,vrgbk357-l4x96ey9-yl4pomew-a0jdyzn8,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d8b98e5b-60a2-4bb0-8e66-a0528ecaaf7c,d8b98e5b-60a2-4bb0-8e66-a0528ecaaf7c,HOST_FEE,NaN,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,DEBIT,Host Fee,2023-12-31T22:46:58.352Z,None,False,False,NaN,False,NaN,Debit,USD,-50,Amount,USD,-50,Amount,USD,-50,Amount,USD,-50,Amount,USD,0,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004,Open Source Collective,opensource,False,ORGANIZATION,Organization,a47byg9n-xozdp8n7-nlmpmjlv-03rek5w8,False,True,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,CONTRIBUTION,NaN,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,CREDIT,Monthly financial contribution to Logseq (Back...,2023-12-31T22:46:58.403Z,ch_3OTXgHBYycQg1OMf0IfKN8Sw,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,n4gx0bro-v5m96nkd-3ay6d8lk-3ey7jzwa,744511,Sanket Sharma,sanket-sharma,False,INDIVIDUAL,Individual,8k03reyd-5agmq5r7-rlyplbwo-z7j4nxv9,True,True,True,TransactionPermi

# Reconciling Wise

In [38]:
df_wise_reconciliation = df_wise.copy()
print(df_wise_reconciliation.info())
df_wise_reconciliation.head()

<class 'pandas.core.frame.DataFrame'>
Index: 3463 entries, 0 to 0
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3463 non-null   int64  
 1   user                   3463 non-null   int64  
 2   targetAccount          3463 non-null   int64  
 3   sourceAccount          3427 non-null   object 
 4   quote                  0 non-null      object 
 5   quoteUuid              3463 non-null   object 
 6   status                 3463 non-null   object 
 7   reference              3463 non-null   object 
 8   rate                   3463 non-null   float64
 9   created                3463 non-null   object 
 10  business               3463 non-null   int64  
 11  transferRequest        0 non-null      object 
 12  details.reference      3463 non-null   object 
 13  hasActiveIssues        3463 non-null   bool   
 14  sourceCurrency         3463 non-null   object 
 15  sourceValue 

,id,user,targetAccount,sourceAccount,quote,quoteUuid,status,reference,rate,created,business,transferRequest,details.reference,hasActiveIssues,sourceCurrency,sourceValue,targetCurrency,targetValue,customerTransactionId
0,575093811,6421549,298725929,59807023,None,be48f661-368c-430b-82f9-94a101390e59,outgoing_payment_sent,,1.00000,2023-01-02 00:07:08,4556280,None,,False,GBP,46.86,GBP,46.86,72d112b5-775a-416e-a0d0-df6219c9d708
0,575919595,6421549,219325936,59807023,None,21416ea6-87fd-4bbd-8a64-2082c6894abc,outgoing_payment_sent,,1.00000,2023-01-02 23:47:59,4556280,None,,False,USD,1550.00,USD,1550.00,9e7ca373-3eb6-45a2-9403-9e9d9f786410
0,575919769,6421549,333432475,59807023,None,5f8c9bc9-0e3e-4748-9ef1-945507d73bb9,outgoing_payment_sent,,0.93705,2023-01-02 23:48:25,4556280,None,,False,USD,71.79,EUR,67.27,f4d246b7-5c51-4430-9698-54e8750fd3c9
0,575919989,6421549,333432571,59807023,None,778efd01-6d36-4b01-994a-253d2c064655,outgoing_payment_sent,,0.93720,2023-01-02 23:49:02,4556280,None,,False,USD,500.00,EUR,468.60,b2abdbb0-2f61-4dc4-bad5-bd04cd0cdaa6
0,575920084,6421549,266354361,59807023,None,59ff5aba-aa30-4fed-ad78-4160aca5dc60,outgoing_payment_sent,,1.00000,2023-01-02 23:49:16,4556280,None,,False,USD,265.00,USD,265.00,1841e526-f886-42d4-9cd4-c1afc5d017e9


In [51]:
df_platform_wise = df_platform[(df_platform['kind'] == 'EXPENSE') & (df_platform['payoutMethod.type'] == 'BANK_ACCOUNT')]
print(df_platform_wise.info())
df_platform_wise.head()

<class 'pandas.core.frame.DataFrame'>
Index: 3483 entries, 3986 to 819753
Data columns (total 73 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    3483 non-null   object 
 1   uuid                                  3483 non-null   object 
 2   kind                                  3483 non-null   object 
 3   paymentMethod                         0 non-null      float64
 4   payoutMethod                          0 non-null      float64
 5   giftCardEmitterAccount                0 non-null      float64
 6   group                                 3483 non-null   object 
 7   type                                  3483 non-null   object 
 8   description                           3483 non-null   object 
 9   createdAt                             3483 non-null   object 
 10  merchantId                            3431 non-null   object 
 11  isRefunded       

,id,uuid,kind,paymentMethod,payoutMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,payoutMethod.type,payoutMethod.name,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
3986,b3978b93-6791-474d-91e8-2ee20a19caa7,b3978b93-6791-474d-91e8-2ee20a19caa7,EXPENSE,NaN,NaN,NaN,e344e7cb-9e21-4368-89ac-86c7af1ca89c,DEBIT,Repo and module support,2023-12-28T17:09:36.200Z,914831281,False,False,NaN,False,NaN,Debit,USD,-12000,Amount,USD,-12000,Amount,USD,-12039,Amount,USD,-12039,Amount,USD,-39,Amount,dgm9bnk8-0437xqr3-kw5qvzeo-ljdayw5r,46356.0,dbatools,dbatools,False,COLLECTIVE,Collective,rmvrwng4-kj03dpb3-b8l6z57o-yl9e8xba,46358,Shawn Melton,wsmelton,False,INDIVIDUAL,Individual,8k03reyd-5agmq5rz-dyyplbwo-z7j4nxv9,False,False,False,TransactionPermissions,NaN,NaN,NaN,NaN,vjrkx5lm-nv904qjz-eanp8bwa-7zdygoe3,INVOICE,Repo and module support,,[],rmvrwng4-kj03dpb3-b8l6z57o-yl9e8xba,Shawn Melton,wsmelton,INDIVIDUAL,Individual,Expense,NaN,BANK_ACCOUNT,NaN,NaN,NaN,NaN,NaN
3987,89ac51ae-2ff1-4786-88a3-052deedd71fe,89ac51ae-2ff1-4786-88a3-052deedd71fe,EXPENSE,NaN,NaN,NaN,101aef0d-4c40-4924-95b6-48b4ed506103,DEBIT,Developer gift,2023-12-28T17:08:55.206Z,914901258,False,False,NaN,False,NaN,Debit,USD,-221386,Amount,USD,-221386,Amount,USD,-222281,Amount,USD,-222281,Amount,USD,-895,Amount,vjrkx5lm-nv904qjl-9lz68bwa-7zdygoe3,30588.0,Stellarium,stellarium,False,COLLECTIVE,Collective,53kzxy4v-07wlr6mk-rk7qmj9n-o8agdbe5,30590,Georg Zotti,georg-zotti,False,INDIVIDUAL,Individual,vrgbk357-l4x96eyv-bj4pomew-a0jdyzn8,False,False,False,TransactionPermissions,NaN,NaN,NaN,NaN,5ax8emk7-lnw54q7g-mgdqgyvj-0o93zdrb,INVOICE,Developer gift,,[],53kzxy4v-07wlr6mk-rk7qmj9n-o8agdbe5,Georg Zotti,georg-zotti,INDIVIDUAL,Individual,Expense,NaN,BANK_ACCOUNT,NaN,NaN,NaN,NaN,NaN
3994,3985e9cb-6e7e-4b56-806e-c8623c909db9,3985e9cb-6e7e-4b56-806e-c8623c909db9,EXPENSE,NaN,NaN,NaN,4ff8a423-6ac4-47ad-bf6b-d97875c4c01c,DEBIT,Yii 3 and yiiframework.com development,2023-12-28T16:35:15.005Z,914833331,False,False,NaN,False,NaN,Debit,USD,-146400,Amount,USD,-146400,Amount,USD,-146439,Amount,USD,-146439,Amount,USD,-39,Amount,eeng0kzd-yvor4pz8-ggbpbma8-37xlw95j,30649.0,Yii Software,yiisoft,False,COLLECTIVE,Collective,rxg0j35l-zkwm6v5x-layqvoe9-8n47daby,451148,Alexey Rogachev,alexey-rogachev,False,INDIVIDUAL,Individual,eng0kzdy-vor4pzev-5lgpbma8-37xlw95j,False,False,False,TransactionPermissions,NaN,NaN,NaN,NaN,vjrkx5lm-nv904qjz-e7np8bwa-7zdygoe3,INVOICE,Yii 3 and yiiframework.com development,,[],rxg0j35l-zkwm6v5x-layqvoe9-8n47daby,Alexey Rogachev,alexey-rogachev,INDIVIDUAL,Individual,Expense,NaN,BANK_ACCOUNT,NaN,NaN,NaN,NaN,NaN
4008,1ea98914-c16d-48b9-ba66-80cd18df2d4c,1ea98914-c16d-48b9-ba66-80cd18df2d4c,EXPENSE,NaN,NaN,NaN,c38b0530-a4aa-4799-ac6d-8cc68ca11cab,

In [50]:
merged_df = pd.merge(df_wise_reconciliation, df_platform_wise[['merchantId', 'id']].dropna(subset=['merchantId']).astype({'merchantId': 'int64'}), left_on='id', right_on='merchantId', how='left', suffixes=('', '_platform_transaction'))

# Drop the extra 'merchantId' column from the merge
# merged_df.drop(columns='merchantId', inplace=True)

# Assign the resulting dataframe back to df_stripe
df_wise_reconciliation = merged_df

print(f"Wise transactions that have a corresponding platform transaction: {len(df_wise_reconciliation[df_wise_reconciliation['id_platform_transaction'].notnull()])}")
print(f"Wise transactions without a corresponding platform transaction: {len(df_wise_reconciliation[df_wise_reconciliation['id_platform_transaction'].isnull()])}")
print(f"Percentage of Wise transactions that have a corresponding platform transaction: {round(len(df_wise_reconciliation[df_wise_reconciliation['id_platform_transaction'].notnull()])/len(df_wise_reconciliation)*100, 2)}%")

df_wise_reconciliation.head()

Wise transactions that have a corresponding platform transaction: 3403
Wise transactions without a corresponding platform transaction: 60
Percentage of Wise transactions that have a corresponding platform transaction: 98.27%


,id,user,targetAccount,sourceAccount,quote,quoteUuid,status,reference,rate,created,business,transferRequest,details.reference,hasActiveIssues,sourceCurrency,sourceValue,targetCurrency,targetValue,customerTransactionId,merchantId,id_platform_transaction
0,575093811,6421549,298725929,59807023,None,be48f661-368c-430b-82f9-94a101390e59,outgoing_payment_sent,,1.00000,2023-01-02 00:07:08,4556280,None,,False,GBP,46.86,GBP,46.86,72d112b5-775a-416e-a0d0-df6219c9d708,575093811.0,94dbd855-92f9-41cf-a47d-2d296aae4eef
1,575919595,6421549,219325936,59807023,None,21416ea6-87fd-4bbd-8a64-2082c6894abc,outgoing_payment_sent,,1.00000,2023-01-02 23:47:59,4556280,None,,False,USD,1550.00,USD,1550.00,9e7ca373-3eb6-45a2-9403-9e9d9f786410,575919595.0,aa44fee3-916a-42eb-915d-a28ef1ebc67a
2,575919769,6421549,333432475,59807023,None,5f8c9bc9-0e3e-4748-9ef1-945507d73bb9,outgoing_payment_sent,,0.93705,2023-01-02 23:48:25,4556280,None,,False,USD,71.79,EUR,67.27,f4d246b7-5c51-4430-9698-54e8750fd3c9,575919769.0,42d3544a-9157-4b98-a340-8b35b31f8378
3,575919989,6421549,333432571,59807023,None,778efd01-6d36-4b01-994a-253d2c064655,outgoing_payment_sent,,0.93720,2023-01-02 23:49:02,4556280,None,,False,USD,500.00,EUR,468.60,b2abdbb0-2f61-4dc4-bad5-bd04cd0cdaa6,575919989.0,0c4091c2-2072-4c49-a7c2-90ae8155744d
4,575920084,6421549,266354361,59807023,None,59ff5aba-aa30-4fed-ad78-4160aca5dc60,outgoing_payment_sent,,1.00000,2023-01-02 23:49:16,4556280,None,,False,USD,265.00,USD,265.00,1841e526-f886-42d4-9cd4-c1afc5d017e9,575920084.0,77b9834e-3989-478a-82e9-540dbf4339cf


In [58]:
df_wise_remainder = df_wise_reconciliation[df_wise_reconciliation['id_platform_transaction'].isnull()]
df_wise_remainder.sample(10)

,id,user,targetAccount,sourceAccount,quote,quoteUuid,status,reference,rate,created,business,transferRequest,details.reference,hasActiveIssues,sourceCurrency,sourceValue,targetCurrency,targetValue,customerTransactionId,merchantId,id_platform_transaction
228,596046428,29739303,59807023,None,None,719ddd54-d03f-4919-b677-73d8b97f5eb7,outgoing_payment_sent,,1.0000,2023-01-29 23:41:09,4556280,None,,False,USD,240000.00,USD,240000.00,719ddd54-d03f-4919-b677-73d8b97f5eb7,NaN,NaN
1022,665203574,6421549,373146940,59807023,None,eef65bf3-68cc-40b8-84bb-61475a161a95,funds_refunded,,1.0000,2023-04-20 15:25:51,4556280,None,,False,USD,2500.00,USD,2500.00,90e751f2-849b-4ab4-833f-f1488cd58065,NaN,NaN
718,639795561,42089850,352322636,59807023,None,126efc32-9929-4dd4-8e6f-49ce7ec855da,outgoing_payment_sent,Correction for expense number 124655,1.0000,2023-03-22 18:41:30,4556280,None,Correction for expense number 124655,False,USD,36.71,USD,36.71,126efc32-9929-4dd4-8e6f-49ce7ec855da,NaN,NaN
1633,720250675,42089850,59807023,None,None,451c2e7d-60b0-421b-a127-02447d66a7c6,outgoing_payment_sent,,1.0000,2023-06-20 16:23:01,4556280,None,,False,USD,200000.00,USD,200000.00,451c2e7d-60b0-421b-a127-02447d66a7c6,NaN,NaN
1030,668545804,6421549,59807023,None,None,1c66d9c2-d8c3-453b-a5b5-201060b37141,outgoing_payment_sent,AutoCredit,1.0000,2023-04-25 06:49:17,4556280,None,AutoCredit,False,USD,114996.48,USD,114996.48,d46dfe3d-9f2c-477a-9f76-bf5782a1c1d1,NaN,NaN
3087,875562410,42089850,59807023,None,None,d034f636-5909-4fec-b16b-723d48282f98,outgoing_payment_sent,,1.0000,2023-11-22 17:47:10,4556280,None,,False,USD,200000.00,USD,200000.00,d034f636-5909-4fec-b16b-723d48282f98,NaN,NaN
2310,790965132,6421549,392402279,59807023,None,ad4f47e7-ab2e-4e0a-a6e4-43b1ce2ccedd,funds_refunded,OC,4.9509,2023-09-01 11:32:19,4556280,None,OC,False,USD,526.41,BRL,2606.21,05b31803-b128-41d3-a6fb-6ff33820c504,NaN,NaN
2714,839365943,42089850,59807023,None,None,e2a4bafa-9ab0-4e44-9ec6-f3ccc6e7d234,outgoing_payment_sent,,1.0000,2023-10-18 19:12:25,4556280,None,,False,USD,200000.00,USD,200000.00,e2a4bafa-9ab0-4e44-9ec6-f3ccc6e7d234,NaN,NaN
1650,723244668,6421549,393787757,59807023,None,e03dc849-c3a0-4d39-97af-00d7f1b58ce5,funds_refunded,OC,4.6765,2023-06-23 16:12:55,4556280,None,OC,False,USD,300.00,MYR,1402.95,803a01c9-3f9f-4005-a32a-1a9a8486c591,NaN,NaN
1813,733974762,42089850,59807023,None,None,23d83bca-5a47-4238-ad0f-30c8b73948ca,outgoing_payment_sent,,1.0000,2023-07-05 02:21:21,4556280,None,,False,USD,140000.00,USD,140000.00,23d83bca-5a47-4238-ad0f-30c8b73948ca,NaN,NaN


# Reconciling Bank

# Reconciling Paypal